In [4]:
import os 
from random import shuffle
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

def image_gen_w_aug(train_parent_directory, test_parent_directory):
    train_datagen =ImageDataGenerator(rescale=1/255,
                                      rotation_range =10,
                                      zoom_range=0.2,
                                      width_shift_range=0.1,
                                      validation_split =0.2)
    test_datagen = ImageDataGenerator(rescale=1/255)
    train_generator = train_datagen.flow_from_directory(train_parent_directory,
                                                        target_size= (75,75),
                                                        batch_size =128,
                                                        class_mode ='categorical',
                                                        subset ='training',
                                                        shuffle=True)
    val_generator = train_datagen.flow_from_directory(train_parent_directory,
                                                        target_size= (75,75),
                                                        batch_size =128,
                                                        class_mode ='categorical',
                                                        subset ='validation')
    test_generator = test_datagen.flow_from_directory(test_parent_directory,
                                                      target_size =(75,75),
                                                      batch_size =32,
                                                      class_mode ='categorical')
    return train_generator, val_generator, test_generator

In [6]:
def model_output_for_TL( pre_trained_model, last_output):
    x = Flatten()(last_output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(pre_trained_model.input, x)
    return model

In [10]:
train_dir = os.path.join('train')
test_dir = os.path.join('valid')

In [11]:
train_generator, validation_generator, test_generator = image_gen_w_aug(train_dir, test_dir)


Found 1200 images belonging to 10 classes.
Found 300 images belonging to 10 classes.
Found 500 images belonging to 10 classes.


In [12]:
pre_trained_model = InceptionV3(input_shape = (75, 75, 3), 
                                include_top = False, 
                                weights = 'imagenet')



87910968/87910968 [==============================] - 18s 0us/step


In [13]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [14]:
last_layer = pre_trained_model.get_layer('mixed5')
last_output = last_layer.output

In [15]:
model_TL = model_output_for_TL(pre_trained_model, last_output)
model_TL.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
history_TL = model_TL.fit(
      train_generator,
      steps_per_epoch=10,  
      epochs=10,
      verbose=1,
      validation_data = validation_generator)


Epoch 1/10
10/10 [==============================] - 3s 360ms/step - loss: 0.3018 - accuracy: 0.9150 - val_loss: 1.3172 - val_accuracy: 0.5533
Epoch 2/10
10/10 [==============================] - 3s 320ms/step - loss: 0.3128 - accuracy: 0.9125 - val_loss: 1.3079 - val_accuracy: 0.5733
Epoch 3/10
10/10 [==============================] - 3s 300ms/step - loss: 0.2991 - accuracy: 0.9175 - val_loss: 1.3431 - val_accuracy: 0.5867
Epoch 4/10
10/10 [==============================] - 3s 294ms/step - loss: 0.2544 - accuracy: 0.9383 - val_loss: 1.3462 - val_accuracy: 0.5967
Epoch 5/10
10/10 [==============================] - 3s 299ms/step - loss: 0.2473 - accuracy: 0.9325 - val_loss: 1.3350 - val_accuracy: 0.5900
Epoch 6/10
10/10 [==============================] - 3s 337ms/step - loss: 0.2448 - accuracy: 0.9275 - val_loss: 1.3643 - val_accuracy: 0.5967
Epoch 7/10
10/10 [==============================] - 3s 330ms/step - loss: 0.2404 - accuracy: 0.9417 - val_loss: 1.3239 - val_accuracy: 0.5833
Epoch 

In [20]:
tf.keras.models.save_model(model_TL,'best_model.hdf5')

C:\Users\tmachine\AppData\Local\Temp\ipykernel_14156\1400139875.py:1: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model_TL,'best_model.hdf5')


In [21]:
test_TL = model_TL.predict(test_generator)

16/16 [==============================] - 1s 44ms/step


In [22]:
print(test_generator[0])
print(test_TL[0])

(array([[[[0.227451  , 0.28627452, 0.29803923],
         [0.19607845, 0.2392157 , 0.21568629],
         [0.14509805, 0.18431373, 0.18823531],
         ...,
         [0.02745098, 0.03529412, 0.03137255],
         [0.02745098, 0.03137255, 0.04705883],
         [0.07843138, 0.08627451, 0.07450981]],

        [[0.13333334, 0.14117648, 0.12156864],
         [0.09803922, 0.10588236, 0.09411766],
         [0.09019608, 0.1137255 , 0.1137255 ],
         ...,
         [0.07843138, 0.07843138, 0.07843138],
         [0.0627451 , 0.05490196, 0.06666667],
         [0.07450981, 0.06666667, 0.07058824]],

        [[0.1137255 , 0.10588236, 0.10980393],
         [0.09411766, 0.09411766, 0.09411766],
         [0.09019608, 0.09803922, 0.08627451],
         ...,
         [0.01960784, 0.02352941, 0.00392157],
         [0.03921569, 0.04313726, 0.0509804 ],
         [0.0509804 , 0.04313726, 0.05490196]],

        ...,

        [[0.8117648 , 0.7254902 , 0.4901961 ],
         [0.76470596, 0.64705884, 0.36078432